<a href="https://colab.research.google.com/github/DavidS256/Parciales/blob/main/script_parcial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Universidad de los Andes**

HE2 IA aplicada a la economía

Parcial 3

**Integrantes:**
*   Marcelo Yepes
*   Juan Diego Barrios
*   David Sandino
*   Loek Kleyn
